## Run cell-based matching

#### Standard imports

In [ ]:
import hpmcm
import glob
import os
import numpy as np
import matplotlib.pyplot as plt

#### Set up the configuration

In [ ]:
DATADIR = "shear_tract"
shear_st = "0p02"
shear = 0.02
shear_type = "gauss"

tract = 10463
SOURCE_TABLEFILES = sorted(glob.glob(os.path.join(DATADIR, f"shear_{shear_type}_{shear_st}_uncleaned_{tract}_*.pq")))
SOURCE_TABLEFILES.reverse()
VISIT_IDS = np.arange(len(SOURCE_TABLEFILES))

REF_DIR = (37.91729, 6.94389)  # RA, DEC in deg
PIXEL_R2CUT = 4.
PIXEL_MATCH_SCALE = 1

#### Make the matcher, reduce the data

In [ ]:
matcher = hpmcm.ShearMatch.createShearMatch(pixelR2Cut=PIXEL_R2CUT, pixelMatchScale=PIXEL_MATCH_SCALE, deshear=-1*shear)

In [ ]:
matcher.reduceData(SOURCE_TABLEFILES, VISIT_IDS)

#### This should have made 200 x 200 cells

In [ ]:
matcher.nCell

#### Run the data

In [ ]:
do_partial = True
if do_partial:
    xRange = range(50, 70)
    yRange = range(170, 190)
    #xRange = [55]
    #yRange = [170]
    matcher.analysisLoop(xRange, yRange)
else:
    matcher.analysisLoop()

#### Show a single cell

In [ ]:
cell = matcher.cellDict[matcher.getCellIdx(50, 170)]
od = cell.analyze(None, 4)
_ = plt.imshow(od['countsMap'], origin='lower')

#### Show a single cluster

In [ ]:
cluster = list(cell.clusterDict.values())[0]
_ = hpmcm.viz_utils.showCluster(od['image'], cluster, cell)

In [ ]:
stats = matcher.extractStats()
shear_stats = matcher.extractShearStats()
obj_shear = shear_stats[1]

In [ ]:
cluster.dist2

#### Get the offsets between the cluster centroid and the sources

In [ ]:
def get_offsets(matcher):
    n = 0
    dd = {
        0:dict(dx=[], dy=[], x=[], y=[]), 
        1:dict(dx=[], dy=[], x=[], y=[]), 
        2:dict(dx=[], dy=[], x=[], y=[]), 
        3:dict(dx=[], dy=[], x=[], y=[]), 
        4:dict(dx=[], dy=[], x=[], y=[]), 
    }
    for cellData in matcher.cellDict.values():
        n += len(cellData.data[0])
        for obj in cellData.objectDict.values():
            if not obj.nUnique == 5 and obj.nSrc == 5:
                continue
            for iCat in range(5):
                mask = obj.catIndices == iCat
                if mask.sum() == 0:
                    continue
                for dx, dy in zip((obj.xPix[mask] - obj.xCent), (obj.yPix[mask] - obj.yCent)):
                    dd[iCat]["dx"].append(dx)
                    dd[iCat]["dy"].append(dy)
                    dd[iCat]["x"].append(float(obj.data[mask].iloc[0].xCell))
                    dd[iCat]["y"].append(float(obj.data[mask].iloc[0].yCell))

    for i in range(5):
        dd[i]['dx'] = np.array(dd[i]['dx'])
        dd[i]['dy'] = np.array(dd[i]['dy'])
        dd[i]['x'] = np.array(dd[i]['x'])
        dd[i]['y'] = np.array(dd[i]['y'])
    print(n)
    return dd                  
                    
    

In [ ]:
dd = get_offsets(matcher)

#### Plots the residuals, they should be flat

In [ ]:
_ = plt.scatter(dd[4]['x'], dd[4]['dx'])

#### Look at how the sources lie within the cells

In [ ]:
_ = plt.hist(matcher.fullData[0].xCellCoadd, bins=np.linspace(-100, 100, 201))
#_ = plt.hist(matcher.fullData[0].loc[stats[0][mask_0].id].xCell_coadd, bins=np.linspace(-100, 100, 201))

In [ ]:
_ = plt.hist(matcher.fullData[0].yCellCoadd, bins=np.linspace(-100, 100, 201))
#_ = plt.hist(matcher.fullData[0].loc[stats[0][mask_0].id].yCell_coadd, bins=np.linspace(-100, 100, 201))

#### Classify the objects by match type

In [ ]:
objLists = matcher.classifyObjects(SNRCut=3.)
hpmcm.Match.printObjectTypes(objLists)

In [ ]:
n_good = len(objLists['ideal'])
bad_list = ['edge_mixed', 'edge_missing', 'edge_extra', 'orphan', 'missing', 'two_missing', 'many_missing', 'extra', 'caught']
n_bad = np.sum([len(objLists[x]) for x in bad_list])

In [ ]:
n_good/(n_good+n_bad)

#### Classify the clusters by match type

In [ ]:
clusterLists = matcher.classifyClusters(SNRCut=30.)
hpmcm.Match.printClusterTypes(clusterLists)

#### Display a few objects

In [ ]:
_ = hpmcm.viz_utils.showShearObjs(matcher, clusterLists['ideal'][5])

In [ ]:
_ = hpmcm.viz_utils.showShearObj(matcher, objLists['missing'][0])

In [ ]:
_ = hpmcm.viz_utils.showShearObj(matcher, objLists['many_missing'][0])

In [ ]:
all_stats = matcher.extractStats()

In [ ]:
shear_stats = matcher.extractShearStats()

In [ ]:
all_stats[0]

In [ ]:
mask = np.bitwise_and(all_stats[3].nUniques != 5, all_stats[3].cellIdx == 10172)
#mask = all_stats[3].cellIdx == 10172
all_stats[0]

In [ ]:
shear_stats[1][mask]